In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers.core import Flatten,Dense,Dropout
from keras.layers.convolutional import Convolution2D,MaxPooling2D,ZeroPadding2D
from keras.optimizers import gradient_descent_v2 


import cv2

In [2]:
df=pd.read_csv('../input/fer2013/fer2013.csv')

In [3]:
X_train,train_y,X_test,test_y=[],[],[],[]

In [4]:
for index,row in df.iterrows():
    val=row['pixels'].split(" ")
    try:
        if 'Training' in row['Usage']:
            X_train.append(np.array(val,'float32'))
            train_y.append(row['emotion'])
        elif 'PublicTest' in row['Usage']:
            X_test.append(np.array(val,'float32'))
            test_y.append(row['emotion'])
    except:
        print(f"error occured at index:{index} and row:{row}")


In [14]:
num_features=64
num_labels=7
batch_size=64
epochs=80
width,height=48,48

In [6]:
X_train=np.array(X_train,'float32')
train_y=np.array(train_y,'float32')
X_test=np.array(X_test,'float32')
test_y=np.array(test_y,'float32')

In [7]:
from tensorflow.keras.utils import to_categorical


In [8]:
train_y=to_categorical(train_y,num_classes = num_labels)
test_y=to_categorical(test_y,num_classes=num_labels)

In [9]:
X_train -=np.mean(X_train,axis=0)
X_train /= np.std(X_train, axis=0)

X_test -= np.mean(X_test, axis=0)
X_test /=np.std(X_test, axis=0)

X_train=X_train.reshape(X_train.shape[0],48,48,1)
X_test=X_test.reshape(X_test.shape[0],48,48,1)

In [10]:
from tensorflow.keras import layers
from tensorflow import keras

from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, Activation
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
from tensorflow.keras.losses import categorical_crossentropy


In [11]:
model=Sequential()
model.add(Conv2D(64,kernel_size=(3,3),activation='relu',input_shape=(X_train.shape[1:])))
model.add(Conv2D(64,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Dropout(0.5))
model.add(Conv2D(64,kernel_size=(3,3),activation='relu'))
model.add(Conv2D(64,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Dropout(0.5))
model.add(Conv2D(128,kernel_size=(3,3),activation='relu'))
model.add(Conv2D(128,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Flatten())
model.add(Dense(1024,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_labels,activation='softmax'))

In [12]:
model.compile(loss=categorical_crossentropy, optimizer='sgd',metrics=['accuracy'])

In [15]:
model.fit(X_train,train_y,
        batch_size=batch_size,
        epochs=epochs,
        verbose=1,
        validation_data=(X_test,test_y),
        shuffle=True)

In [17]:
fer_json=model.to_json()
with open("fer.json","w") as json_file:
    json_file.write(fer_json)
model.save_weights("fer.h5")    

In [18]:
for layer in model.layers:
    weights = layer.get_weights()
print(weights)  

In [19]:
import os
from keras.models import model_from_json
from keras.preprocessing import image

In [20]:
model=model_from_json(open("fer.json",'r').read())
model.load_weights('fer.h5')
face_haar_cascade=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [75]:
test_image=cv2.imread('../input/face-reco/Zomato.jpg')



In [76]:
plt.imshow(cv2.cvtColor(test_image,cv2.COLOR_BGR2RGB))

In [77]:
face_cascade=cv2.CascadeClassifier('../input/haarcascades/haarcascade_eye.xml')
from keras.preprocessing.image import img_to_array
plt.imshow(cv2.cvtColor(test_image,cv2.COLOR_BGR2RGB))

In [78]:
gray_image=cv2.cvtColor(test_image,cv2.COLOR_BGR2GRAY)

In [79]:
faces = face_cascade.detectMultiScale(gray_image, 1.1, 4)
for (x, y, w, h) in faces:
    cv2.rectangle(test_image, (x,y), (x+w,y+h), (255,0,0)) 
    roi_gray=gray_image[y:y+w,x:x+h]
    roi_gray=cv2.resize(roi_gray,(48,48))
    image_pixels=img_to_array(roi_gray)
    image_pixels=np.expand_dims(image_pixels,axis=0)
    image_pixels/=255
    predictions=model.predict(image_pixels)
    max_index=np.argmax(predictions[0])
    emotion_detection= ('angry', 'disgust', 'Fear', 'happy', 'sad', 'surprise', 'neutral')
    emotion_prediction=emotion_detection[max_index]
    print(emotion_prediction)
    font=cv2.FONT_HERSHEY_SIMPLEX
    org=(50,50)
    fontScale=1
    color=(255,0,0)
    thickness=2
    image=cv2.putText(test_image,emotion_prediction,org,font,fontScale,color,thickness,cv2.LINE_AA)
    plt.imshow(cv2.cvtColor(image,cv2.COLOR_BGR2RGB))